# Reading and processing

In [ ]:
import numpy as np
from datetime import timedelta  # Not actually used, but shows a nice Ploomber feature
from random import choice

import pandas as pd
import pytorch_lightning as pl
import torch as t
import neuralforecast as nf

## Read csv

In [ ]:
# Data can be downloaded here: https://ourworldindata.org/covid-cases
df = pd.read_csv("~/Downloads/owid-covid-data.csv", parse_dates=['date'])
df.head(3)

In [ ]:
# Columns to be used in the model as extra information
X_cols = ["human_development_index", "gdp_per_capita", "stringency_index", 
          "life_expectancy", "hospital_beds_per_thousand"]
# Column to predict
Y_col =  "new_cases"

## Cleaning and converting into desired format

In [ ]:
# These names are required by the library
column_renamer = {"location": "unique_id",
                  "date": "ds",
                  "new_cases": "y"}

In [ ]:
# Library requires that all series end at the same time. Not many countries are lost this way
df = df[df.groupby("location")["date"].transform(max) == df["date"].max()]
minmaxes = df.groupby("location")["date"].agg([min, max, "count"])
minmaxes["proportions"] = (((minmaxes["max"] - minmaxes["min"]).dt.days + 1) / minmaxes["count"])
minmaxes[minmaxes["proportions"] != 1]

In [ ]:
# Aggregating data to be at a weekly level since daily was very messy
grouper_aggregator = {col: "mean" for col in X_cols}
grouper_aggregator.update({Y_col: "sum"})
df = df.groupby(["location", 
                 pd.Grouper(key='date', freq='W-MON')]).agg(grouper_aggregator).reset_index()

## Creating DataFrame for Y

In [ ]:
Y_df = df[["location", "date", Y_col]]
Y_df = Y_df.rename(columns=column_renamer).fillna(0)
Y_df.sample(3)

## Creating DataFrame for X

In [ ]:
X_df = df[["location", "date"] + X_cols]
X_df = X_df.rename(columns=column_renamer).fillna(0)
X_df.sample(3)

## Split Test and Train

In [ ]:
output_size = 6  # This is how many periods you wish predict. 6 weeks in this case
Y_df_test = Y_df.groupby('unique_id').tail(output_size)
Y_df_train = Y_df.drop(Y_df_test.index)

X_df_train = X_df.drop(Y_df_test.index)

In [ ]:
# Since I want to show a comparable result along with my predictions,
# output split is to be doubled so that I can have train  >  validation  >  predictions
#                                                         (output_size1)   (output_size2)
input_size = 2 * output_size

## Train

In [ ]:
# This library tool tags the data as train and validation. See graph below
train_mask_df, val_mask_df, _ = nf.experiments.utils.get_mask_dfs(
    Y_df=Y_df_train,
    ds_in_val=output_size,
    ds_in_test=0
)
train_mask_df.sample(3)

## Plot of train/test proportion

In [ ]:
plot_df = Y_df_train.merge(
    train_mask_df.drop('available_mask', axis=1).rename(columns={'sample_mask': 'sample_mask_train'}),
    how='left',
    on=['unique_id', 'ds']
).merge(
    val_mask_df.drop('available_mask', axis=1).rename(columns={'sample_mask': 'sample_mask_val'}),
    how='left',
    on=['unique_id', 'ds']
)

In [ ]:
plot_df['y_train'] = np.where(plot_df['sample_mask_train'] == 1, plot_df['y'], np.nan)
plot_df['y_val'] = np.where(plot_df['sample_mask_val'] == 1, plot_df['y'], np.nan)
plot_df.sample(3)

In [ ]:
plot_df.query('unique_id == "Argentina"').set_index('ds')[['y_train', 'y_val']].plot(title="Train vs Validation")

## Converting into WindowsDataset

In [ ]:
# This is an optimized window object made by Nixtla. It can handle time series more efficiently
train_dataset = nf.data.tsdataset.WindowsDataset(
    Y_df=Y_df_train, 
    X_df=X_df_train,
     f_cols=X_cols,
    input_size=input_size,
    output_size=output_size,
    mask_df=train_mask_df
)

In [ ]:
# This is an optimized window object made by Nixtla. It can handle time series more efficiently
val_dataset = nf.data.tsdataset.WindowsDataset(
    Y_df=Y_df_train, 
     X_df=X_df_train,
     f_cols=X_cols,
    input_size=input_size,
    output_size=output_size,
    mask_df=val_mask_df
)

## Converting into TimeSeriesLoader

In [ ]:
# This is another object made by Nixtla. It's used to give the WindowsDatasets to the models
train_loader = nf.data.tsloader.TimeSeriesLoader(
    train_dataset, batch_size=32, 
    n_windows=256,
    shuffle=True
)

In [ ]:
# This is another object made by Nixtla. It's used to give the WindowsDatasets to the models
val_loader = nf.data.tsloader.TimeSeriesLoader(
    val_dataset, 
    batch_size=1
)

## Instancing the model

In [ ]:
model = nf.models.nbeats.nbeats.NBEATS(
    n_time_in=input_size,
    n_time_out=output_size,
    n_x=len(X_cols),  # Number of X cols you provided
    n_x_hidden=[len(X_cols), 10], # These are neural network hidden layers for X variables
    frequency='W-MON', 
    seasonality=4  # Since they are weeks. It would be 7 in case of daily or any other number if you know what you are doing
)

## Training the model

In [ ]:
early_stopping = pl.callbacks.EarlyStopping(monitor="val_loss")

trainer = pl.Trainer(max_epochs=50,  # The bigger this number, the deeper the model is
                     gpus=-1 if t.cuda.is_available() else 0,
                     callbacks=[early_stopping])

trainer.fit(model, train_loader, val_loader)

## Forecasting using the model

In [ ]:
Y_df_forecast = model.forecast(Y_df_train, X_df=X_df)
Y_df_forecast.rename(columns={'y': 'y_hat'}, inplace=True)
Y_df_forecast.head()

# Results

## Random country plot

In [ ]:
Y_df_plot = Y_df_test.append(Y_df_train).merge(Y_df_forecast, how='left', on=['unique_id', 'ds']).sort_values("ds")
country = choice(Y_df_plot["unique_id"].unique())
Y_df_plot.query(f'unique_id == "{country}"').tail(100).set_index('ds').plot(title=country)